In [96]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag

In [107]:
%run ./utils.ipynb

In [20]:
%run ./relation_extraction.ipynb

In [90]:
text, relations = extract_relations(
    "parsed/race/train/middle/2549.txt.tree"
)

In [117]:
def rule_explanation_001(text, relation):
    assert(relation is not None)
    nucleus_relation, nucleus_segment = relation.get_first_nucleus()
    satellite_relation, _ = relation.get_satellite()
    search_in = [satellite_relation]
#     if nucleus_relation is not None:
#         if nucleus_relation.type != "Background":
#             search_in.append(nucleus_relation)
#         else:
#             search_in.append(nucleus_relation.left_child)
#             search_in.append(nucleus_relation.right_child)
    for relation in search_in:
        bg_rel, depth = find_nested_relation(
            "Background", relation
        )
        if bg_rel is not None:
            break
    if bg_rel is None:
        return None, None
    else:
        _, bg_nucleus_segment = bg_rel.get_first_nucleus()
        _, bg_satellite_segment = bg_rel.get_satellite()
        bg_nucleus_text = trim_connective(
            clean(
                text[bg_nucleus_segment.start:bg_nucleus_segment.end]
            )
        )
        bg_nucleus_text = clean(
                text[bg_nucleus_segment.start:bg_nucleus_segment.end]
            )
        bg_satellite_text = clean(
            text[bg_satellite_segment.start:bg_satellite_segment.end]
        )
        pt = pos_tag(word_tokenize(bg_satellite_text))
        assert(len(pt) > 0)
        if pt[0][1][0] == "V":
            bg_nucleus_text = "that " + bg_nucleus_text
        nucleus_text=clean(
            text[nucleus_segment.start:nucleus_segment.end]
        )
        statement = "{nucleus} because {bg_nucleus}".format(
            nucleus=remove_trailing_punctuation(nucleus_text),
            bg_nucleus=lowercase_first_letter(bg_nucleus_text)
        )
        if statement[-1] not in punctuation:
            statement += "."
        if len(sent_tokenize(statement)) == 1:
            return statement, depth
        else:
            return None, None

In [93]:
if "Explanation" in relations:
    for relation in relations["Explanation"]:
        print(rule_explanation_001(text, relation))
        print("")

They get up about 4:50 in the morning because they are working.

None

None

None



In [ ]:
# def generate_explanation_statement(text, relation):
#     if relation.left.type == "N":
#         assert(relation.right.type == "S")
#         nucleus = text[relation.left.start:relation.left.end]
#         satellite = text[relation.right.start:relation.right.end]
#     else:
#         assert(relation.left.type == "S")
#         assert(relation.right.type == "N")
#         nucleus = text[relation.left.start:relation.left.end]
#         satellite = text[relation.right.start:relation.right.end]
    
#     statement = (
# f"""``{satellite}''\nTHIS CAN EXPLAIN THE FOLLOWING FACT:
#   (*) {nucleus}
# """
#     )
    
#     return clean_statement(statement)

In [6]:
DIRECTORY = "parsed/race/train/middle"

def find_text():
    for file_name in os.listdir(DIRECTORY):
        path = os.path.join(DIRECTORY, file_name)
        text, relations = extract_relations(path)

        if "Explanation" in relations:
            for relation in relations["Explanation"]:
                if find_nested_relation("Background", relation):
                    return path

In [7]:
find_text()

'parsed/race/train/high/18576.txt.tree'

In [118]:
DIRECTORY = "parsed/race/train/middle"

statements = []
for file_name in os.listdir(DIRECTORY):
    path = os.path.join(DIRECTORY, file_name)
    text, relations = extract_relations(path)
    
    if "Explanation" in relations:
        for relation in relations["Explanation"]:
            statement, depth = rule_explanation_001(
                text, 
                relation
            )
            if statement is not None:
                statements.append(f"[{path}, {depth}]\n{statement}")

In [119]:
with open(
    "statements/explanation/train/middle.txt", 
    "wt"
) as f:
    f.write("\n".join(statements))